# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm
from linear_operator import settings

import pyro
import math
import pickle
import time
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
import arviz as az
import seaborn as sns

In [2]:
import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [3]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('Data/X_test.csv', header=None, delimiter=',').values

Y_train_8 = pd.read_csv('Data/Y_train_8.csv', header=None, delimiter=',').values
Y_test_8 = pd.read_csv('Data/Y_test_8.csv', header=None, delimiter=',').values

Y_train_20 = pd.read_csv('Data/Y_train_20.csv', header=None, delimiter=',').values
Y_test_20 = pd.read_csv('Data/Y_test_20.csv', header=None, delimiter=',').values

Y_train_std = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',').values
Y_test_std = pd.read_csv('Data/Y_test_std.csv', header=None, delimiter=',').values

In [4]:
train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_8 = torch.tensor(Y_train_8, dtype=torch.float32)
test_y_8 = torch.tensor(Y_test_8, dtype=torch.float32)

train_y_20 = torch.tensor(Y_train_20, dtype=torch.float32)
test_y_20 = torch.tensor(Y_test_20, dtype=torch.float32)

train_y = torch.tensor(Y_train_std, dtype=torch.float32)
test_y = torch.tensor(Y_test_std, dtype=torch.float32)

## PCA of output

In [5]:
pca_8 = PCA(n_components = 8)
pca_20 = PCA(n_components = 20)

pca_8.fit(Y_train_std)
pca_20.fit(Y_train_std)

PCA(n_components=20)

# Emulators

In [6]:
Device = 'cpu'

In [7]:
row_idx = 0

input_point = test_y[row_idx,:]
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = 100)


## Training part

### LocalGP

In [ ]:
def train_and_predict_LocalGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):

    LocalGP_models, LocalGP_likelihoods = Training.train_one_row_LocalGP(
        train_x, train_y, test_y, row_idx, k_num=K_num, lr=0.05, num_iterations=5000, patience=10, device=Device
    )
    
    preds = Prediction.full_preds(LocalGP_models, LocalGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).cpu().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    return preds




In [ ]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_LocalGP)(row_idx, train_x, train_y_20, test_x, test_y_20, PCA_trans = pca_20) for row_idx in range(test_y.shape[0]))
full_test_preds_LocalGP = np.vstack(results)

MSE_LocalGP = np.mean((full_test_preds_LocalGP - test_y.numpy()) ** 2)

### MultiGP

In [ ]:
row_idx = 0

input_point = test_y[row_idx,:]
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = 100)


MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP(local_train_x, local_train_y, n_tasks = local_train_y.shape[1], 
                                                                                 covar_type = 'RQ', lr=0.05, num_iterations=5000, patience=10, device=Device)


In [ ]:
row_idx = 0

input_point = test_y_20[row_idx,:]
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_20, k = 100)

MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP_lcm(local_train_x, local_train_y, n_tasks = local_train_y.shape[1], 
                                                                                     lr=0.05, num_iterations=5000, patience=10, device=Device)


In [ ]:
def train_and_predict_MGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):


    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)

    MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_MultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], covar_type = 'RQ', 
                                                                                     lr=0.05, num_iterations=10000, patience=10, device=Device)

    preds = Prediction.preds_for_one_model(MultitaskGP_models, MultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).squeeze().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    return preds



In [ ]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_MGP)(row_idx, train_x, train_y, test_x, test_y) for row_idx in range(test_y.shape[0]))
full_test_preds_MGP = np.vstack(results)


MSE_MGP = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)

In [ ]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_MGP)(row_idx, train_x, train_y_20, test_x, test_y_20, PCA_trans = pca_20) for row_idx in range(test_y.shape[0]))
full_test_preds_MGP = np.vstack(results)


MSE_MGP_20 = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)

### NN + MultiGP

In [ ]:
row_idx = 0

input_point = test_y[row_idx,:]
local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = 100)

MultitaskGP_models, MultitaskGP_likelihoods = Training.train_one_row_NNMultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                            feature_extractor_class = FeatureE.FeatureExtractor_1, covar_type = 'RBF', 
                                                                                            lr=0.05, num_iterations=5000, patience=10, device = Device)
    

In [ ]:
def train_and_predict_NNMGP(row_idx, train_x, train_y, test_x, test_y, K_num = 100, Device = 'cpu', PCA_trans = 'None'):


    input_point = test_y[row_idx,:]
    local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y, k = K_num)

    NNMultitaskGP_models, NNMultitaskGP_likelihoods = Training.train_one_row_NNMultitaskGP(local_train_x, local_train_y, n_tasks = train_y.shape[1], 
                                                                                            feature_extractor_class = FeatureE.FeatureExtractor_1, covar_type = 'RBF', 
                                                                                            lr=0.05, num_iterations=5000, patience=10, device = Device)

    preds = Prediction.preds_for_one_model(NNMultitaskGP_models, NNMultitaskGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)).squeeze().detach().numpy()
    if PCA_trans != 'None':
        preds = PCA_trans.inverse_transform(preds)

    return preds



In [ ]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_NNMGP)(row_idx, train_x, train_y, test_x, test_y) for row_idx in range(test_y.shape[0]))
full_test_preds_MGP = np.vstack(results)


MSE_NNMGP = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)

In [ ]:
results = Parallel(n_jobs=-1)(delayed(train_and_predict_NNMGP)(row_idx, train_x, train_y_20, test_x, test_y_20, PCA_trans = pca_20) for row_idx in range(test_y.shape[0]))
full_test_preds_MGP = np.vstack(results)


MSE_NNMGP_20 = np.mean((full_test_preds_MGP - test_y.numpy()) ** 2)

### VGP

In [ ]:
inducing_points = train_x[:600, :].to(Device)
VGP_models, VGP_likelihoods = Training.train_full_VGP_Parallel(train_x, train_y_20, inducing_points, covar_type = 'RQ', lr=0.025, num_iterations=5000, patience=30, device=Device)


### MVGP

In [8]:
MVGP_models, MVGP_likelihoods = Training.train_full_MultitaskVGP(train_x, train_y_20, num_latents=8, num_inducing=500, lr_hyper=0.05, lr_variational=0.05, num_iterations=5000, patience=20, device=Device)


  9%|▉         | 462/5000 [07:06<1:09:54,  1.08it/s, loss=-8.26]  


In [ ]:
full_test_preds_MVGP = Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, test_x.to(Device)).cpu().detach().numpy()
np.mean((full_test_preds_MVGP - test_y_20.numpy()) ** 2)

In [ ]:
full_test_preds_MVGP = pca_20.inverse_transform(full_test_preds_MVGP)
MSE_MVGP = np.mean((full_test_preds_MVGP - test_y.numpy()) ** 2)
MSE_MVGP

### DGP

In [ ]:
# Device = 'cuda'
DGP_2 = Training.train_full_DGP_2(train_x, train_y, num_hidden_dgp_dims = 20, inducing_num = 400, num_iterations = 5000, patiences = 50, device=Device)

## Estimation

### Point estimation

In [ ]:
bounds = bound.get_bounds(local_train_x)

estimated_params, func_loss = Estimation.multi_start_estimation(MultitaskGP_models, MultitaskGP_likelihoods, row_idx, test_y, bounds, Estimation.estimate_params_for_one_model_Adam, 
                                                                num_starts=5, num_iterations=2000, lr=0.01, patience=10, 
                                                                attraction_threshold=0.1, repulsion_strength=0.1, device=Device)


# full_estimated_params = estimated_params.detach().numpy()


In [ ]:
estimated_params

### MCMC 

In [ ]:
mcmc_result_Normal = Estimation.run_mcmc_Normal(Prediction.preds_for_one_model, MultitaskGP_models, MultitaskGP_likelihoods, row_idx, test_y, local_train_x, num_sampling=1000, warmup_step=500)

posterior_samples_Normal = mcmc_result_Normal.get_samples()

param_names = [f'param_{i}' for i in range(len(bounds))]

posterior_means_array = np.zeros(len(param_names))


for idx, param_name in enumerate(param_names):
    samples = posterior_samples_Normal[param_name]
    if samples.ndim > 1:
        samples = samples.reshape(-1)
    mean_value = torch.mean(samples).item()
    posterior_means_array[idx] = mean_value

In [27]:
bounds = bound.get_bounds(local_train_x)
mcmc_result_Normal = Estimation.run_mcmc_Normal(Prediction.preds_for_one_model, MVGP_models, MVGP_likelihoods, row_idx, test_y_20, local_train_x, num_sampling=1000, warmup_step=500)

posterior_samples_Normal = mcmc_result_Normal.get_samples()

param_names = [f'param_{i}' for i in range(len(bounds))]

posterior_means_array = np.zeros(len(param_names))


for idx, param_name in enumerate(param_names):
    samples = posterior_samples_Normal[param_name]
    if samples.ndim > 1:
        samples = samples.reshape(-1)
    mean_value = torch.mean(samples).item()
    posterior_means_array[idx] = mean_value

Warmup:   0%|          | 7/1500 [00:02,  1.75it/s, step size=6.07e-02, acc. prob=0.725]

KeyboardInterrupt: 

In [53]:
pca_20.inverse_transform(test_y_20)

tensor([[-9.3573e-01,  6.1779e-01,  6.8070e-01,  ...,  8.5060e-01,
          6.2020e-01,  7.9584e-01],
        [ 8.5607e-02, -1.2016e+00, -1.2601e+00,  ...,  5.9026e-01,
          4.3521e-01,  5.6560e-01],
        [ 9.2254e-02,  6.0058e-01,  6.1135e-01,  ..., -1.2402e+00,
         -1.0344e+00, -1.1633e+00],
        ...,
        [-2.7734e-01, -3.9543e-01, -5.5892e-01,  ...,  9.1329e-01,
          1.0037e+00,  1.1026e+00],
        [-6.8586e-01,  8.1831e-01,  9.0375e-01,  ...,  6.0887e-02,
          3.9767e-03, -5.7555e-03],
        [ 2.8117e+00, -1.2590e+00, -1.1005e+00,  ..., -3.7310e+00,
         -4.4462e+00, -3.7701e+00]], dtype=torch.float64)

In [ ]:
Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, test_x[:1,:]).detach()

tensor([[-4.2929e+00, -4.9929e-01,  1.7102e+00,  1.0454e+00,  1.4452e+00,
         -2.6576e-01,  1.5284e-02, -2.9460e-01,  2.7067e-02, -6.9303e-03,
          9.1465e-03,  1.3538e-02, -6.2384e-03, -7.7724e-03, -6.4912e-04,
         -4.5905e-03, -2.9543e-04,  8.4651e-03, -3.6628e-04, -1.3892e-04]])

In [72]:
pca_20.inverse_transform(Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, test_x[:1,:]).detach())

tensor([[-0.9378,  0.6452,  0.6560,  1.0159, -0.0039, -0.0332,  0.4220,  0.6563,
          0.6867,  0.9951, -0.1319,  0.0967,  0.5482,  1.0062,  0.9915,  0.8291,
          1.0115,  0.8437, -0.5972, -0.7443, -0.8944, -0.0084,  0.0468, -0.3653,
         -0.6139, -1.0057, -0.9965,  0.0714, -0.0187, -0.5161, -0.8947, -0.8651,
         -0.9111, -0.9466, -0.8400,  0.6936,  0.8185,  0.6498, -0.0163, -0.0388,
          0.2892,  0.7627,  1.0401,  1.1166, -0.0171, -0.1110,  0.3812,  0.8260,
          0.6623,  0.8186,  0.7164,  0.7761]], dtype=torch.float64)

In [79]:
components = torch.from_numpy(pca_20.components_).to(dtype=torch.float32)
mean = torch.from_numpy(pca_20.mean_).to(dtype=torch.float32)
torch.matmul(Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, test_x[:1,:]), components) + mean

tensor([[-0.9378,  0.6452,  0.6560,  1.0159, -0.0039, -0.0332,  0.4220,  0.6563,
          0.6867,  0.9951, -0.1319,  0.0967,  0.5482,  1.0062,  0.9915,  0.8291,
          1.0115,  0.8437, -0.5972, -0.7443, -0.8944, -0.0084,  0.0468, -0.3653,
         -0.6139, -1.0057, -0.9965,  0.0714, -0.0187, -0.5161, -0.8947, -0.8651,
         -0.9111, -0.9466, -0.8400,  0.6936,  0.8185,  0.6498, -0.0163, -0.0388,
          0.2892,  0.7627,  1.0401,  1.1166, -0.0171, -0.1110,  0.3812,  0.8260,
          0.6623,  0.8186,  0.7164,  0.7761]], grad_fn=<AddBackward0>)

In [80]:
def run_mcmc_Normal_pca(Pre_function, Models, Likelihoods, PCA_Function, row_idx, test_y, local_train_x, num_sampling=2000, warmup_step=1000):
    def model():
        params = []
        
        for i in range(local_train_x.shape[1]):
            mean = local_train_x[:, i].mean()
            std = local_train_x[:, i].std()
            param_i = pyro.sample(f'param_{i}', dist.Normal(mean, std))
            params.append(param_i)
        
        theta = torch.stack(params)
        
        sigma = pyro.sample('sigma', dist.HalfNormal(10.0))

        components = torch.from_numpy(PCA_Function.components_).to(dtype=torch.float32)
        mean_PCA = torch.from_numpy(PCA_Function.mean_).to(dtype=torch.float32)
        
        preds = Pre_function(Models, Likelihoods, theta.unsqueeze(0))
        mu_value = (torch.matmul(preds, components) + mean_PCA).squeeze()
        
        y_obs = test_y[row_idx, :]
        
        pyro.sample('obs', dist.Normal(mu_value, sigma), obs=y_obs)

    nuts_kernel = NUTS(model)
    mcmc = MCMC(nuts_kernel, num_samples=num_sampling, warmup_steps=warmup_step)
    mcmc.run()

    
    return mcmc



In [81]:
run_mcmc_Normal_pca(Prediction.preds_for_one_model, MVGP_models, MVGP_likelihoods, pca_20, row_idx, test_y, local_train_x, num_sampling=1000, warmup_step=500)


Warmup:   1%|          | 18/1500 [00:08,  1.59it/s, step size=3.43e-02, acc. prob=0.785]

KeyboardInterrupt: 